In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
 
df = pd.read_excel('MVO Assignment.xlsx')
tickers = df['Unnamed: 1'].dropna()
param = list(df['Unnamed: 5'].dropna())
weight_client  = df['Unnamed: 2'].dropna()

rf = param[3]
rf_per = rf/12

print(tickers)
print(weight_client)
print(param)


1     M
2    SU
3    RY
4     F
5     A
Name: Unnamed: 1, dtype: object
1    0.2
2    0.2
3    0.2
4    0.2
5    0.2
Name: Unnamed: 2, dtype: float64
[datetime.datetime(2015, 11, 1, 0, 0), datetime.datetime(2023, 1, 1, 0, 0), '1mo', 0.03, 1]


In [2]:
import yfinance as yf

history = pd.DataFrame()

for ticker in tickers:
    yf_ticker = yf.Ticker(ticker)
    #history[ticker] = yf_ticker.history(period=param[0], interval = param[1], auto_adjust = True)['Close']
    history[ticker] = yf_ticker.history(interval=param[2], start = param[0],end=param[1], auto_adjust = True)['Close']

    col_shift = ticker + '_shifted'
    history[col_shift] = history[ticker].shift(periods = 1,fill_value = 0, axis = 0)

    col_ret = ticker + '_returns'
    history[col_ret] = (history[ticker] - history[col_shift])/history[col_shift]

    history = history.drop(columns=[col_shift, ticker])


history = history.tail(-1)
mu_vector = []
for i,ticker in enumerate(tickers):
    col_ret = ticker + '_returns'
    mean = history[col_ret].mean()
    col_de_mean = ticker + '_de_meaned'
    history[col_de_mean] = history[col_ret] - mean

    mu_vector.append(mean) 
    
history.index = history.index.tz_localize(None)

print(mu_vector)
print(history)



[0.009617600053607674, 0.010618066649555512, 0.010478934343563694, 0.006877547679464797, 0.0180878663288682]
            M_returns  SU_returns  RY_returns  F_returns  A_returns  \
Date                                                                  
2015-12-01  -0.104913   -0.057749   -0.057520  -0.016748  -0.000239   
2016-01-01   0.166316   -0.087209   -0.043673  -0.152590  -0.097035   
2016-02-01   0.069290    0.039490    0.006655   0.083075  -0.007968   
2016-03-01   0.020366    0.136029    0.129292   0.079137   0.066935   
2016-04-01  -0.094690    0.065277    0.079221   0.004444   0.026850   
...               ...         ...         ...        ...        ...   
2022-08-01  -0.018697   -0.047731   -0.035862   0.037440  -0.041930   
2022-09-01  -0.095266   -0.129022   -0.032244  -0.257762  -0.052242   
2022-10-01   0.342955    0.235353    0.026988   0.193750   0.138215   
2022-11-01   0.127098   -0.044199    0.090266   0.039641   0.122141   
2022-12-01  -0.121277   -0.034682   -0.

In [3]:
import openpyxl
import math
import numpy as np
import scipy.stats 

wb = openpyxl.load_workbook('MVO Assignment.xlsx', data_only=True)
sheet = wb['Report']

sheet.delete_cols(1, 100000)
sheet.delete_rows(1, 100000)

sheet.cell(column=1, row=2).value = 'Client Portfolio Stocks:'
sheet.cell(column=1, row=3).value = 'Stock Average Return:'
sheet.cell(column=1, row=4).value = 'Stock Standard Deviation:'
sheet.cell(column=1, row=5).value = 'Stock Kurtosis:'
sheet.cell(column=1, row=6).value = 'Stock Skew:'


for i,ticker in enumerate(tickers):
    col_ret = ticker + '_returns'

    sheet.cell(column=i+2, row=2).value = ticker
    sheet.cell(column=i+2, row=3).value = np.mean(history[col_ret])
    sheet.cell(column=i+2, row=4).value = np.std(history[col_ret])
    sheet.cell(column=i+2, row=5).value = scipy.stats.kurtosis (history[col_ret])
    sheet.cell(column=i+2, row=6).value = scipy.stats.skew (history[col_ret])
 
    history = history.drop(columns=[col_ret])


wb.save('MVO Assignment.xlsx')

In [4]:
import numpy as np

# (Epsilon transpose * Epsilon )/ (n-1)
covariance = np.matmul(np.transpose(history),history)/(len(history) - 1)

covariance

,0,1,2,3,4
M_de_meaned,0.024821,0.010402,0.003997,0.007990,0.002590
SU_de_meaned,0.010402,0.012091,0.003849,0.005968,0.001846
RY_de_meaned,0.003997,0.003849,0.003085,0.003143,0.002062
F_de_meaned,0.007990,0.005968,0.003143,0.011350,0.003140
A_de_meaned,0.002590,0.001846,0.002062,0.003140,0.004650


In [5]:
#Question 1

import math
weight_client_mat_transpose = np.asmatrix(weight_client)
covariance = np.asmatrix(covariance)
weight_client_mat = np.asmatrix(np.transpose(weight_client_mat_transpose))


current_client_stddev = math.sqrt(np.matmul(np.matmul(weight_client_mat_transpose,covariance), weight_client_mat))
current_client_exp_ret = np.matmul(mu_vector,weight_client_mat)
current_client_sharpe_ratio = (current_client_exp_ret-rf_per)/current_client_stddev

current_client_stddev_ann = current_client_stddev * math.sqrt(12)
current_client_exp_ret_ann = 12 * current_client_exp_ret
current_client_sharpe_ratio_ann = (current_client_exp_ret_ann-rf)/current_client_stddev_ann

#output to excel

sheet.cell(column=1, row=8).value = 'Current client portfolio metrics:'
sheet.cell(column=2, row=8).value = 'Periodic'
sheet.cell(column=3, row=8).value = 'Annual'

sheet.cell(column=1, row=9).value = 'Expected Returns:'
sheet.cell(column=1, row=10).value = 'Annualized Standard Deviation'
sheet.cell(column=1, row=11).value = 'Annualized Sharpe Ratio'

sheet.cell(column=2, row=9).value = float(current_client_exp_ret)
sheet.cell(column=2, row=10).value = float(current_client_stddev)
sheet.cell(column=2, row=11).value = float(current_client_sharpe_ratio)

sheet.cell(column=3, row=9).value = float(current_client_exp_ret_ann)
sheet.cell(column=3, row=10).value = float(current_client_stddev_ann)
sheet.cell(column=3, row=11).value = float(current_client_sharpe_ratio_ann)

for i,ticker in enumerate(tickers):
    sheet.cell(column=6+i, row=8).value = ticker
    sheet.cell(column=6+i, row=9).value = weight_client_mat.tolist()[i][0]



wb.save('MVO Assignment.xlsx')



In [6]:
#Question 2a

a_mat = 2*covariance

a_mat_arr = np.array(a_mat)

num_rows, num_cols = a_mat_arr.shape

new_column = np.ones((num_rows, 1), dtype=int)
a_mat_arr = np.hstack((a_mat_arr, new_column))

new_row = np.ones((1, num_cols + 1), dtype=int) 
a_mat_arr = np.vstack((a_mat_arr, new_row))

a_mat_arr[-1, -1] = 0

muvec_arr = np.array(mu_vector)

muvec_arr = np.append(muvec_arr, [0])

a_mat_ep_arr = np.vstack((a_mat_arr, muvec_arr))

muvec_arr = np.append(muvec_arr, [0])
a_mat_ep_arr = np.hstack((a_mat_ep_arr, np.transpose(np.asmatrix(muvec_arr))))


a_mat_ep_arr

b_vec_ep = np.zeros(num_cols, dtype=int)
b_vec_ep = np.append(b_vec_ep, [1, float(current_client_exp_ret)])

weight_ep = np.matmul(np.linalg.inv(a_mat_ep_arr),b_vec_ep)

weight_ep = weight_ep[:, :-2]

##

weight_ep_mat_transpose = np.asmatrix(weight_ep)
weight_ep_mat = np.asmatrix(np.transpose(weight_ep_mat_transpose))

ep_stddev = math.sqrt(np.matmul(np.matmul(weight_ep_mat_transpose,covariance), weight_ep_mat))
ep_exp_ret = np.matmul(mu_vector,weight_ep_mat)
ep_sharpe_ratio = (ep_exp_ret-rf_per)/ep_stddev

ep_stddev_ann = ep_stddev * math.sqrt(12)
ep_exp_ret_ann = 12 * ep_exp_ret
ep_sharpe_ratio_ann = (ep_exp_ret_ann-rf)/ep_stddev_ann

#output to excel

sheet.cell(column=1, row=18).value = 'Efficient portfolio (minimize variance) metrics:'
sheet.cell(column=2, row=18).value = 'Periodic'
sheet.cell(column=3, row=18).value = 'Annual'

sheet.cell(column=1, row=19).value = 'Expected Returns:'
sheet.cell(column=1, row=20).value = 'Standard Deviation'
sheet.cell(column=1, row=21).value = 'Sharpe Ratio'

sheet.cell(column=2, row=19).value = float(ep_exp_ret)
sheet.cell(column=2, row=20).value = float(ep_stddev)
sheet.cell(column=2, row=21).value = float(ep_sharpe_ratio)

sheet.cell(column=3, row=19).value = float(ep_exp_ret_ann)
sheet.cell(column=3, row=20).value = float(ep_stddev_ann)
sheet.cell(column=3, row=21).value = float(ep_sharpe_ratio_ann)

for i,ticker in enumerate(tickers):
    sheet.cell(column=6+i, row=18).value = ticker
    sheet.cell(column=6+i, row=19).value = weight_ep_mat.tolist()[i][0]

wb.save('MVO Assignment.xlsx')


In [7]:
#Question 3

weights_orp = np.matmul(np.linalg.inv(covariance) ,np.transpose((np.asmatrix(mu_vector) - rf_per)))
ones =  np.ones(num_cols, dtype=int)
divisor = np.matmul(ones,weights_orp)

weights_orp=weights_orp/divisor
 

weights_orp


##

weight_orp_mat = np.asmatrix(weights_orp)
weight_orp_mat_transpose = np.asmatrix(np.transpose(weight_orp_mat))

orp_stddev = math.sqrt(np.matmul(np.matmul(weight_orp_mat_transpose,covariance), weight_orp_mat))
orp_exp_ret = np.matmul(mu_vector,weight_orp_mat)
orp_sharpe_ratio = (orp_exp_ret-rf_per)/orp_stddev

orp_stddev_ann = orp_stddev * math.sqrt(12)
orp_exp_ret_ann = 12 * orp_exp_ret
orp_sharpe_ratio_ann = (orp_exp_ret_ann-rf)/orp_stddev_ann

#output to excel

sheet.cell(column=1, row=23).value = 'Optimum Risky Portfolio (on frontier) metrics:'
sheet.cell(column=2, row=23).value = 'Periodic'
sheet.cell(column=3, row=23).value = 'Annual'

sheet.cell(column=1, row=24).value = 'Expected Returns:'
sheet.cell(column=1, row=25).value = 'Standard Deviation'
sheet.cell(column=1, row=26).value = 'Sharpe Ratio'

sheet.cell(column=2, row=24).value = float(orp_exp_ret)
sheet.cell(column=2, row=25).value = float(orp_stddev)
sheet.cell(column=2, row=26).value = float(orp_sharpe_ratio)

sheet.cell(column=3, row=24).value = float(orp_exp_ret_ann)
sheet.cell(column=3, row=25).value = float(orp_stddev_ann)
sheet.cell(column=3, row=26).value = float(orp_sharpe_ratio_ann)

for i,ticker in enumerate(tickers):
    sheet.cell(column=6+i, row=23).value = ticker
    sheet.cell(column=6+i, row=24).value = weight_orp_mat.tolist()[i][0]

wb.save('MVO Assignment.xlsx')



In [8]:
#part 2a


max_ret_orp_line_ann = orp_sharpe_ratio_ann*current_client_stddev_ann + rf # y = mx + b
max_ret_orp_line = orp_sharpe_ratio*current_client_stddev + rf_per # y = mx + b

min_sd_orp_line_ann = (current_client_exp_ret_ann - rf) / orp_sharpe_ratio_ann # x = y-b / m
min_sd_orp_line = (current_client_exp_ret - rf_per) / orp_sharpe_ratio # x = y-b / m

dist_orp = math.sqrt(  float(orp_stddev_ann)* float(orp_stddev_ann)+ float(orp_exp_ret_ann- rf)*float(orp_exp_ret_ann- rf) )

dist_port1 = math.sqrt(  float(current_client_stddev_ann)* float(current_client_stddev_ann)+ float(max_ret_orp_line_ann- rf)*float(max_ret_orp_line_ann- rf) )
dist_port2 = math.sqrt(  float(min_sd_orp_line_ann)* float(min_sd_orp_line_ann)+ float(current_client_exp_ret_ann- rf)*float(current_client_exp_ret_ann- rf) )


weight_port_max_ret = (dist_port1/dist_orp)
weight_port_min_sd = (dist_port2/dist_orp)

weight_tot_max_ret = weight_port_max_ret * weight_orp_mat
weight_tot_min_sd = weight_port_min_sd * weight_orp_mat

sheet.cell(column=6, row=28).value = 'Risk Free'
sheet.cell(column=6, row=29).value = 1 - weight_port_max_ret

sheet.cell(column=6, row=33).value = 'Risk Free'
sheet.cell(column=6, row=34).value = 1 - weight_port_min_sd



for i,ticker in enumerate(tickers):
    sheet.cell(column=7+i, row=28).value = ticker
    sheet.cell(column=7+i, row=29).value = weight_tot_max_ret.tolist()[i][0]

for i,ticker in enumerate(tickers):
    sheet.cell(column=7+i, row=33).value = ticker
    sheet.cell(column=7+i, row=34).value = weight_tot_min_sd.tolist()[i][0]



#output to excel

sheet.cell(column=1, row=28).value = 'Optimal Portfolio (max returns):'
sheet.cell(column=2, row=28).value = 'Periodic'
sheet.cell(column=3, row=28).value = 'Annual'

sheet.cell(column=1, row=29).value = 'Expected Returns:'
sheet.cell(column=1, row=30).value = 'Standard Deviation'
sheet.cell(column=1, row=31).value = 'Sharpe Ratio'

sheet.cell(column=2, row=29).value = float(max_ret_orp_line)
sheet.cell(column=2, row=30).value = float(current_client_stddev)
sheet.cell(column=2, row=31).value = float(orp_sharpe_ratio)

sheet.cell(column=3, row=29).value = float(max_ret_orp_line_ann)
sheet.cell(column=3, row=30).value = float(current_client_stddev_ann)
sheet.cell(column=3, row=31).value = float(orp_sharpe_ratio_ann)


sheet.cell(column=1, row=33).value = 'Optimal Portfolio (min standard deviation):'
sheet.cell(column=2, row=33).value = 'Periodic'
sheet.cell(column=3, row=33).value = 'Annual'

sheet.cell(column=1, row=34).value = 'Expected Returns:'
sheet.cell(column=1, row=35).value = 'Standard Deviation'
sheet.cell(column=1, row=36).value = 'Sharpe Ratio'

sheet.cell(column=2, row=34).value = float(current_client_exp_ret)
sheet.cell(column=2, row=35).value = float(min_sd_orp_line)
sheet.cell(column=2, row=36).value = float(orp_sharpe_ratio)

sheet.cell(column=3, row=34).value = float(current_client_exp_ret_ann)
sheet.cell(column=3, row=35).value = float(min_sd_orp_line_ann)
sheet.cell(column=3, row=36).value = float(orp_sharpe_ratio_ann)

wb.save('MVO Assignment.xlsx')

In [9]:
#Question 4

weights_gmv = np.matmul(np.linalg.inv(covariance) ,np.transpose(np.asmatrix(ones)))

divisor = np.matmul(np.asmatrix(ones) ,weights_gmv)

weights_gmv = weights_gmv / divisor


weight_gmv_mat = np.asmatrix(weights_gmv)
weight_gmv_mat_transpose = np.asmatrix(np.transpose(weight_gmv_mat))


gmv_stddev = math.sqrt(np.matmul(np.matmul(weight_gmv_mat_transpose,covariance), weight_gmv_mat))
gmv_exp_ret = np.matmul(mu_vector,weight_gmv_mat)
gmv_sharpe_ratio = (gmv_exp_ret-rf_per)/gmv_stddev

gmv_stddev_ann = gmv_stddev * math.sqrt(12)
gmv_exp_ret_ann = 12 * gmv_exp_ret
gmv_sharpe_ratio_ann = (gmv_exp_ret_ann-rf)/gmv_stddev_ann

#output to excel

sheet.cell(column=1, row=13).value = 'Global Min Variance portfolio metrics:'
sheet.cell(column=2, row=13).value = 'Periodic'
sheet.cell(column=3, row=13).value = 'Annual'

sheet.cell(column=1, row=14).value = 'Expected Returns:'
sheet.cell(column=1, row=15).value = 'Standard Deviation'
sheet.cell(column=1, row=16).value = 'Sharpe Ratio'

sheet.cell(column=2, row=14).value = float(gmv_exp_ret)
sheet.cell(column=2, row=15).value = float(gmv_stddev)
sheet.cell(column=2, row=16).value = float(gmv_sharpe_ratio)

sheet.cell(column=3, row=14).value = float(gmv_exp_ret_ann)
sheet.cell(column=3, row=15).value = float(gmv_stddev_ann)
sheet.cell(column=3, row=16).value = float(gmv_sharpe_ratio_ann)

for i,ticker in enumerate(tickers):
    sheet.cell(column=6+i, row=13).value = ticker
    sheet.cell(column=6+i, row=14).value = weight_gmv_mat.tolist()[i][0]



wb.save('MVO Assignment.xlsx')




In [10]:
import openpyxl
wb = openpyxl.load_workbook('MVO Assignment.xlsx', data_only=True)
sheet = wb['Output']

sheet.delete_cols(1, 100000)
sheet.delete_rows(1, 100000)


for i,ticker in enumerate(tickers):
    sheet.cell(column=2+i, row=2).value = ticker

sheet.cell(column=1, row=2).value = 'Ticker'



for i,mu in enumerate(mu_vector):
    sheet.cell(column=2+i, row=3).value = mu
sheet.cell(column=1, row=3).value = 'Exp. Ret.'


sheet.cell(column=2, row=6).value = current_client_stddev
sheet.cell(column=1, row=6).value = 'Std. Dev. (Client Portfolio)'

sheet.cell(column=1, row=8).value = 'Cov. Mat.'



cov_arr = np.asarray(covariance)

for i, a in enumerate(cov_arr.tolist()):
    for j, b in enumerate(a):
        sheet.cell(column=int(i+2), row=int(j+8)).value = b



wb.save('MVO Assignment.xlsx')
